# 전력 수요량 예측 프로젝트 (2025 봄학기 시계열데이터분석 중간보고서)

**✅ 조원 이름: (여기에 조원 이름 작성)**

## 1. 시계열 데이터에 대한 설명

본 프로젝트는 한국전력거래소의 '시간별 전국 전력수요량' 데이터를 활용하였다. 데이터는 날짜별로 1시부터 24시까지 시간별 전력 사용량(MW)을 기록하며, 이를 long 포맷으로 변환하고 datetime 인덱스를 추가하였다.

다변량 시계열 구성을 위해 시간(hour), 요일(dayofweek), 월(month) 정보를 추가로 파생하였다.

## 2. 데이터 탐색 및 분석 포인트

시간대별, 요일별, 월별 전력 사용량의 변화를 분석하였다.

- 시간대별: 출근시간대, 퇴근시간대 전력 수요 상승
- 요일별: 주말 대비 평일에 전력 수요 높음
- 월별: 겨울철 전력 사용량 증가

pandas와 matplotlib를 사용하여 시각화하였다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 데이터 불러오기
file_path = "한국전력거래소_시간별 전국 전력수요량_20241231.csv"
df = pd.read_csv(file_path, encoding="cp949")

# wide -> long 변환
df_long = df.melt(id_vars=["날짜"], var_name="시간", value_name="전력수요(MW)")
df_long.loc[df_long["시간"] == "24시", "시간"] = "00시"
df_long.loc[df_long["시간"] == "00시", "날짜"] = pd.to_datetime(df_long.loc[df_long["시간"] == "00시", "날짜"]) + pd.Timedelta(days=1)
df_long["날짜"] = pd.to_datetime(df_long["날짜"])
df_long["datetime"] = pd.to_datetime(df_long["날짜"].dt.strftime('%Y-%m-%d') + " " + df_long["시간"], format="%Y-%m-%d %H시")
df_long = df_long.sort_values("datetime").set_index("datetime")[["전력수요(MW)"]]

# 다변량 특성 추가
df_long["시간"] = df_long.index.hour
df_long["요일"] = df_long.index.dayofweek
df_long["월"] = df_long.index.month

df_long.head()

In [ ]:
# 시간대별, 요일별, 월별 평균 전력수요 시각화
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

hourly_avg = df_long.groupby("시간")["전력수요(MW)"].mean()
weekday_avg = df_long.groupby("요일")["전력수요(MW)"].mean()
monthly_avg = df_long.groupby("월")["전력수요(MW)"].mean()

axs[0].plot(hourly_avg.index, hourly_avg.values, marker='o')
axs[0].set_title("시간대별 평균 전력수요")
axs[0].set_xlabel("시간")
axs[0].grid(True)

axs[1].plot(weekday_avg.index, weekday_avg.values, marker='o', color='orange')
axs[1].set_title("요일별 평균 전력수요")
axs[1].set_xlabel("요일 (0=월요일)")
axs[1].grid(True)

axs[2].plot(monthly_avg.index, monthly_avg.values, marker='o', color='green')
axs[2].set_title("월별 평균 전력수요")
axs[2].set_xlabel("월")
axs[2].grid(True)

plt.tight_layout()
plt.show()

## 3. 시계열 모델 적용

Darts의 RegressionModel과 sklearn의 RandomForest를 이용하여 시간, 요일, 월 변수를 사용해 전력수요 예측을 수행하였다.

In [ ]:
from darts import TimeSeries
from darts.models import RegressionModel
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from sklearn.ensemble import RandomForestRegressor

# 타겟 시계열
target_series = TimeSeries.from_series(df_long["전력수요(MW)"])

# 다변량 외부 변수
covariates = pd.concat([
    datetime_attribute_timeseries(target_series, attribute="hour"),
    datetime_attribute_timeseries(target_series, attribute="day_of_week"),
    datetime_attribute_timeseries(target_series, attribute="month")
], axis=1)

covariates_series = TimeSeries.from_dataframe(covariates)

# 모델 정의 및 학습
model = RegressionModel(lag_target=[-1, -2, -3, -24], lags_covariates=[0], model=RandomForestRegressor())
model.fit(series=target_series, past_covariates=covariates_series)

# 48시간 예측
forecast = model.predict(n=48)

# 결과 시각화
target_series.plot(label="실제값")
forecast.plot(label="예측값")
plt.title("48시간 전력수요 예측 (Darts + RandomForest)")
plt.legend()
plt.grid(True)
plt.show()

## 4. 의미 해석 및 가치 도출

전력 수요는 시간대, 요일, 월별로 뚜렷한 주기성을 가지며, 다변량 시계열 분석을 통해 효과적으로 예측할 수 있었다.

특히 RandomForest 기반 RegressionModel은 단기 예측에 강점을 보였으며, 향후 기온, 공휴일 등 외부 변수를 추가하면 더 높은 예측 정확도를 기대할 수 있다.